<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Train-test-split" data-toc-modified-id="Train-test-split-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Train-test split</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fit the model</a></span></li></ul></div>

In [149]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pickle

In [150]:
df = pd.read_csv('df_model_eu.csv')

In [151]:
df.head()

,Unnamed: 0,age_18-21,age_22-24,age_25-29,age_30-34,age_35-39,age_40-44,age_45-49,age_50-54,age_55-59,...,team_size_g_Large,team_size_g_Medium,team_size_g_No team,team_size_g_Small,role_group_Data Analyst,role_group_Data Engineer,role_group_Data Scientist,role_group_Other,programming_lang_c,salary_avg
0,10,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,4,12500.0
1,11,0,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,2,2500.0
2,17,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,2,27500.0
3,24,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,2,35000.0
4,35,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,2,500.0


In [152]:
df = df.drop(['Unnamed: 0'], axis=1)

### Train-test split

In [153]:
X = df.drop('salary_avg', axis=1)
y = df['salary_avg']

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [155]:
X_train_num = X_train.select_dtypes(np.number)
X_test_num = X_test.select_dtypes(np.number)

In [16]:
transformer = StandardScaler()
transformer.fit(X_train_num)

# saving in a pickle
with open('std_transformer.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer.pickle', 'rb') as file:
    loaded_transformer = pickle.load(file)

X_train_ = loaded_transformer.transform(X_train_num)
X_test_ = loaded_transformer.transform(X_test_num)

### Fit the model

In [156]:
clf = RandomForestRegressor(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.14183434625530422


In [157]:
from sklearn.model_selection import cross_val_score
clf = RandomForestRegressor(max_depth=2, random_state=0)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

0.11202554546467543


In [107]:
from sklearn.metrics import accuracy_score
scores =[]
for k in range(1, 200):
    rfc = RandomForestRegressor(n_estimators=k)
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

import matplotlib.pyplot as plt
%matplotlib inline

# plot the relationship between K and testing accuracy
plt.plot(x_axis, y_axis)
plt.plot(range(1, 200), scores)
plt.xlabel('Value of n_estimators for Random Forest Classifier')
plt.ylabel('Testing Accuracy')


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [158]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10, 50, 60, 80, 100, 150, 200],
    'min_samples_split': [2, 5, 10],
    #'min_leaf_size': [50, 60, 70],
    'min_samples_leaf' : [1, 3, 2],
    'max_depth': [10, 20, 30, 40, 50],
    'max_features': ['sqrt', 'auto', 'log2']
#    'max_samples' : ['None', 0.5]
    }
clf = RandomForestRegressor(random_state=100)

grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True,n_jobs=-1)
grid_search.fit(X_train,y_train)
grid_search.best_params_ #To check the best set of parameters returned


{'max_depth': 50,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [159]:
clf = RandomForestRegressor(random_state=0, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, max_depth=50, n_estimators=200)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

0.3488503029317752


In [160]:
clf = RandomForestRegressor(random_state=0, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, max_depth=30, n_estimators=200)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.38440498170276505


In [161]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [162]:
from sklearn.metrics import mean_squared_error
t= mean_squared_error(y_test, y_pred, squared=False)


In [163]:
#MSE = np.square(np.subtract(y_actual,y_predicted)).mean() 
import math
RMSE = math.sqrt(t)
print(RMSE)

186.5700032992363


In [ ]:
#h2o 

In [164]:
clf.fit(X_train, y_train)
X_train.head()
feature_names = X_train.columns
feature_names = list(feature_names)

df = pd.DataFrame(list(zip(feature_names, clf.feature_importances_)))
df.columns = ['columns_name', 'score_feature_importance']
df.sort_values(by=['score_feature_importance'], ascending = False)

,columns_name,score_feature_importance
85,programming_lang_c,0.062027
32,country_UK,0.036307
28,country_Russia,0.033189
20,country_Germany,0.028010
57,industry_Accounting/Finance,0.027957
...,...,...
62,industry_Hospitality/Entertainment/Sports,0.001372
67,industry_Military/Security/Defense,0.001291
10,age_70,0.001250
68,industry_Non-profit/Service,0.001167


In [29]:
clf.fit(X_train_, y_train)
X_train_.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'